### Pre-configuration :

In [5]:
from pyspark.sql.functions import explode
from pyspark.sql.functions import split
from pyspark.context import SparkContext
from pyspark.conf import SparkConf
from pyspark.sql.session import SparkSession
from pyspark.sql.functions import col
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DoubleType, BooleanType, TimestampType, DateType
from pyspark.sql.functions import from_json


spark = SparkSession.builder \
    .appName("MyApp") \
    .config("spark.jars.packages", 'org.apache.spark:spark-sql-kafka-0-10_2.12:3.0.0') \
    .config("spark.sql.repl.eagerEval.enabled", True) \
    .getOrCreate()

schema = StructType(
      [
        StructField("name", StringType(), False),
        StructField("price", DoubleType(), False),
        StructField("timestamp", TimestampType(), False),
      ]
    )

kafka_server = "kafka1:9092"   

lines = (spark.readStream                        # Get the DataStreamReader
  .format("kafka")                                 # Specify the source format as "kafka"
  .option("kafka.bootstrap.servers", kafka_server) # Configure the Kafka server name and port
  .option("subscribe", "stock")                       # Subscribe to the "en" Kafka topic 
  .option("startingOffsets", "earliest")           # The start point when a query is started
  .option("maxOffsetsPerTrigger", 100)             # Rate limit on max offsets per trigger interval
  .load()
  .select(from_json(col("value").cast("string"), schema).alias("parsed_value"))
# Load the DataFrame
)
df = lines.select("parsed_value.*")

Be sure to start the stream on Kafka!

Update Pyspark :

In [1]:
!pip install --upgrade pyspark==3.5.0


[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python3 -m pip install --upgrade pip


In [2]:
import pyspark
print(pyspark.__version__)

3.5.0


## 1. Select the N most valuable stocks in a window

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import window, col, desc, max, rank
from pyspark.sql.window import Window

# Define window duration and sliding interval
window_duration = "1 hour"
slide_duration = "1 hour"  # slide the window every hour as well

# Group by window and name, then compute the maximum price
windowed_df = df.groupBy(
    window(col("timestamp"), window_duration, slide_duration),
    col("name")
).agg(max("price").alias("max_value"))

# Define the process for each micro-batch
def process_batch(df, epoch_id):
    # Define a window spec for ranking stocks within each window
    windowSpec = Window.partitionBy("window").orderBy(desc("max_value"))

    # Add rank for each stock within the window
    ranked_df = df.withColumn("rank", rank().over(windowSpec))

    # Filter for top N stocks within each window
    N = 5  #i choose 5
    topN_df = ranked_df.where(col("rank") <= N)

    # Show the result
    topN_df.show()

# Write the top N stocks to console for each window using foreachBatch
query = windowed_df.writeStream \
    .foreachBatch(process_batch) \
    .outputMode("update") \
    .start()

query.awaitTermination()

23/11/06 20:51:13 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.
23/11/06 20:51:13 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-e8ba7ac9-d8c7-4524-9d88-9071692c6f2b. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
23/11/06 20:51:13 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


+--------------------+----+---------+----+
|              window|name|max_price|rank|
+--------------------+----+---------+----+
|{2023-10-25 10:00...| AZO|   384.01|   1|
|{2023-10-25 10:00...|ISRG| 192.6465|   2|
|{2023-10-25 10:00...|  RE|   128.37|   3|
|{2023-10-25 10:00...| PXD|    128.0|   4|
|{2023-10-25 10:00...| CMI|   119.79|   5|
+--------------------+----+---------+----+



+--------------------+----+---------+----+
|              window|name|max_price|rank|
+--------------------+----+---------+----+
|{2023-10-25 10:00...| GWW|   228.67|   1|
|{2023-10-25 10:00...|WYNN|   120.24|   2|
|{2023-10-25 10:00...| PVH|   114.07|   3|
|{2023-10-25 10:00...| BXP|    110.1|   4|
|{2023-10-25 10:00...| FRT|   106.58|   5|
+--------------------+----+---------+----+



+--------------------+-----+---------+----+
|              window| name|max_price|rank|
+--------------------+-----+---------+----+
|{2023-10-25 10:00...|GOOGL| 414.2338|   1|
|{2023-10-25 10:00...|  CMG|    378.4|   2|
|{2023-10-25 10:00...|  GWW|    254.0|   3|
|{2023-10-25 10:00...| EQIX|   219.94|   4|
|{2023-10-25 10:00...| ISRG| 192.6465|   5|
+--------------------+-----+---------+----+



+--------------------+----+---------+----+
|              window|name|max_price|rank|
+--------------------+----+---------+----+
|{2023-10-25 10:00...|PCLN|   819.98|   1|
|{2023-10-25 10:00...|AMZN|   269.98|   2|
|{2023-10-25 10:00...| GWW|   261.34|   3|
|{2023-10-25 10:00...|EQIX|   219.94|   4|
|{2023-10-25 10:00...| IBM|   211.98|   5|
+--------------------+----+---------+----+



...

+--------------------+-----+---------+----+
|              window| name|max_price|rank|
+--------------------+-----+---------+----+
|{2023-10-25 10:00...|GOOGL|   608.89|   1|
|{2023-10-25 10:00...| EQIX|   219.94|   2|
|{2023-10-25 10:00...|  MCK|  191.466|   3|
|{2023-10-25 10:00...| ALXN|    180.2|   4|
|{2023-10-25 10:00...| ORLY|   155.06|   5|
+--------------------+-----+---------+----+



+--------------------+----+---------+----+
|              window|name|max_price|rank|
+--------------------+----+---------+----+
|{2023-10-25 10:00...| CMG|    692.9|   1|
|{2023-10-25 10:00...| PXD|   213.11|   2|
|{2023-10-25 10:00...| IBM|   211.98|   3|
|{2023-10-25 10:00...|WYNN|  183.117|   4|
|{2023-10-25 10:00...|ILMN|    178.0|   5|
+--------------------+----+---------+----+



+--------------------+-----+---------+----+
|              window| name|max_price|rank|
+--------------------+-----+---------+----+
|{2023-10-25 10:00...| PCLN|  1182.56|   1|
|{2023-10-25 10:00...|  CMG|    692.9|   2|
|{2023-10-25 10:00...|GOOGL|   608.89|   3|
|{2023-10-25 10:00...| REGN|   349.83|   4|
|{2023-10-25 10:00...|  GWW|   261.34|   5|
+--------------------+-----+---------+----+



+--------------------+----+---------+----+
|              window|name|max_price|rank|
+--------------------+----+---------+----+
|{2023-10-25 10:00...|REGN|   377.45|   1|
|{2023-10-25 10:00...| ESS|   194.68|   2|
|{2023-10-25 10:00...| PSA|   177.78|   3|
|{2023-10-25 10:00...|  GS|   177.54|   4|
|{2023-10-25 10:00...|COST|   130.54|   5|
+--------------------+----+---------+----+



+--------------------+----+---------+----+
|              window|name|max_price|rank|
+--------------------+----+---------+----+
|{2023-10-25 10:00...| BLK|   348.23|   1|
|{2023-10-25 10:00...| AMG|   203.89|   2|
|{2023-10-25 10:00...| MCK|    201.7|   3|
|{2023-10-25 10:00...|  GS|   190.89|   4|
|{2023-10-25 10:00...| LMT|   188.44|   5|
+--------------------+----+---------+----+



## 2. Select the stocks that lost value between two windows

In [1]:
# M1:

In [ ]:
from pyspark.sql.functions import window, col, last, lag
from pyspark.sql.window import Window
from pyspark.sql import DataFrame

# Function to process each micro-batch
def process_batch(df: DataFrame, epoch_id):
    
    # Define the window specification with partitioning by name and ordering by window
    w = Window.partitionBy("name").orderBy("window")

    # Add new columns with the lagged or previous window last value
    df_with_lag = df.withColumn("prev_last_value", lag("value").over(w)) \
                    .withColumn("previous_window", lag(col("window")).over(w))

    # Calculate the difference between the current last value and the previous last value
    df_with_diff = df_with_lag.withColumn("value_diff", col("value") - col("prev_last_value"))

    # Filter for stocks that have lost value
    stocks_lost_value = df_with_diff.filter(col("value_diff") < 0)

    # Select columns in the specified order
    final_df = stocks_lost_value.select(
        "name",
        "window",
        "previous_window",
        "value",
        "prev_last_value",
        "value_diff"
    )

    # Write the full lines to the console without truncating = just fo better view
    final_df.write.format("console").option("truncate", False).save()

# Define the window duration, i choose "1 minute"
window_duration = "1 minute"

# Group the data by window and stock name and get the last value in each window
windowedLast = df \
    .withWatermark("timestamp", "2 minutes") \
    .groupBy(window(col("timestamp"), window_duration), col("name")) \
    .agg(last(col("price")).alias("value"))  # Use last value instead of avg

# Use foreachBatch to apply the custom logic to each micro-batch
query = windowedLast \
    .writeStream \
    .outputMode("update") \
    .foreachBatch(process_batch) \
    .start()

query.awaitTermination()

23/11/06 19:31:13 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-0b0d2fba-dc45-4b8f-85f1-e6b3b7e042ee. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
23/11/06 19:31:13 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


+----+------+---------------+---------+--------------+----------+
|name|window|previous_window|avg_price|prev_avg_price|price_diff|
+----+------+---------------+---------+--------------+----------+
+----+------+---------------+---------+--------------+----------+



+----+------+---------------+---------+--------------+----------+
|name|window|previous_window|avg_price|prev_avg_price|price_diff|
+----+------+---------------+---------+--------------+----------+
+----+------+---------------+---------+--------------+----------+



+----+------------------------------------------+------------------------------------------+------------------+--------------+-------------------+
|name|window                                    |previous_window                           |avg_price         |prev_avg_price|price_diff         |
+----+------------------------------------------+------------------------------------------+------------------+--------------+-------------------+
|SO  |{2023-10-25 10:13:00, 2023-10-25 10:14:00}|{2023-10-25 10:12:00, 2023-10-25 10:13:00}|47.355000000000004|48.245        |-0.8899999999999935|
+----+------------------------------------------+------------------------------------------+------------------+--------------+-------------------+



+----+------------------------------------------+------------------------------------------+---------+-----------------+-------------------+
|name|window                                    |previous_window                           |avg_price|prev_avg_price   |price_diff         |
+----+------------------------------------------+------------------------------------------+---------+-----------------+-------------------+
|ZBH |{2023-10-25 10:14:00, 2023-10-25 10:15:00}|{2023-10-25 10:13:00, 2023-10-25 10:14:00}|78.77    |81.185           |-2.4150000000000063|
|NEE |{2023-10-25 10:14:00, 2023-10-25 10:15:00}|{2023-10-25 10:13:00, 2023-10-25 10:14:00}|80.33    |81.15            |-0.8200000000000074|
|SIG |{2023-10-25 10:14:00, 2023-10-25 10:15:00}|{2023-10-25 10:13:00, 2023-10-25 10:14:00}|69.01    |70.94999999999999|-1.9399999999999835|
+----+------------------------------------------+------------------------------------------+---------+-----------------+-------------------+



...

+----+------------------------------------------+------------------------------------------+---------+--------------+-------------------+
|name|window                                    |previous_window                           |avg_price|prev_avg_price|price_diff         |
+----+------------------------------------------+------------------------------------------+---------+--------------+-------------------+
|AAPL|{2023-10-25 10:26:00, 2023-10-25 10:27:00}|{2023-10-25 10:25:00, 2023-10-25 10:26:00}|95.48    |97.32         |-1.8399999999999892|
|V   |{2023-10-25 10:26:00, 2023-10-25 10:27:00}|{2023-10-25 10:25:00, 2023-10-25 10:26:00}|53.0975  |54.26         |-1.1625000000000014|
|SRE |{2023-10-25 10:26:00, 2023-10-25 10:27:00}|{2023-10-25 10:25:00, 2023-10-25 10:26:00}|100.81   |103.48        |-2.6700000000000017|
+----+------------------------------------------+------------------------------------------+---------+--------------+-------------------+



+----+------+---------------+---------+--------------+----------+
|name|window|previous_window|avg_price|prev_avg_price|price_diff|
+----+------+---------------+---------+--------------+----------+
+----+------+---------------+---------+--------------+----------+



+----+------+---------------+---------+--------------+----------+
|name|window|previous_window|avg_price|prev_avg_price|price_diff|
+----+------+---------------+---------+--------------+----------+
+----+------+---------------+---------+--------------+----------+



+----+------------------------------------------+------------------------------------------+---------+--------------+-------------------+
|name|window                                    |previous_window                           |avg_price|prev_avg_price|price_diff         |
+----+------------------------------------------+------------------------------------------+---------+--------------+-------------------+
|NWL |{2023-10-25 10:28:00, 2023-10-25 10:29:00}|{2023-10-25 10:27:00, 2023-10-25 10:28:00}|32.66    |33.56         |-0.9000000000000057|
+----+------------------------------------------+------------------------------------------+---------+--------------+-------------------+



+----+------------------------------------------+------------------------------------------+---------+--------------+-------------------+
|name|window                                    |previous_window                           |avg_price|prev_avg_price|price_diff         |
+----+------------------------------------------+------------------------------------------+---------+--------------+-------------------+
|BLK |{2023-10-25 10:29:00, 2023-10-25 10:30:00}|{2023-10-25 10:28:00, 2023-10-25 10:29:00}|347.22   |348.23        |-1.009999999999991 |
|VLO |{2023-10-25 10:29:00, 2023-10-25 10:30:00}|{2023-10-25 10:28:00, 2023-10-25 10:29:00}|49.17    |51.24         |-2.0700000000000003|
+----+------------------------------------------+------------------------------------------+---------+--------------+-------------------+



In [3]:
# M2:

In [ ]:
from pyspark.sql.functions import window, col
from pyspark.sql.functions import udf
from pyspark.sql.types import DoubleType

# Define the window and watermark duration
window_duration = "5 hours"
watermark_duration = "1 hour"

# Apply windowing and watermarking to the DataFrame
windowedStocks = df \
    .withWatermark("timestamp", watermark_duration) \
    .groupBy(
        window(col("timestamp"), window_duration),
        col("name")
    ) \
    .agg(
        {"price": "collect_list"}  # Collects all prices in the window into a list
    ) \
    .select(
        col("name"),
        col("window"),
        col("collect_list(price)").alias("prices")
    )

def price_diff(prices):
    if len(prices) > 1:
        return prices[-1] - prices[0]  # Assuming the prices are in chronological order
    else:
        return 0.0

price_diff_udf = udf(price_diff, DoubleType())

# Apply the UDF to calculate the price difference
stocksPriceDiff = windowedStocks.withColumn("price_diff", price_diff_udf(col("prices")))

# Filter the stocks that lost value
stocksLostValue = stocksPriceDiff.filter(col("price_diff") < 0)

# Output the result to a console sink
query = stocksLostValue \
    .writeStream \
    .outputMode("update") \
    .format("console") \
    .option("truncate", False) \
    .start()

query.awaitTermination()

23/11/05 13:44:59 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-0a311cae-1744-4628-b71c-87364b094d5d. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
23/11/05 13:44:59 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


-------------------------------------------
Batch: 0
-------------------------------------------
+----+------------------------------------------+------------------+--------------------+
|name|window                                    |prices            |price_diff          |
+----+------------------------------------------+------------------+--------------------+
|DFS |{2023-10-25 10:00:00, 2023-10-25 15:00:00}|[39.82, 38.82]    |-1.0                |
|FLS |{2023-10-25 10:00:00, 2023-10-25 15:00:00}|[53.4833, 53.4166]|-0.06669999999999732|
|FITB|{2023-10-25 10:00:00, 2023-10-25 15:00:00}|[16.63, 16.16]    |-0.46999999999999886|
+----+------------------------------------------+------------------+--------------------+



-------------------------------------------
Batch: 1
-------------------------------------------
+----+------------------------------------------+---------------------+--------------------+
|name|window                                    |prices               |price_diff          |
+----+------------------------------------------+---------------------+--------------------+
|VAR |{2023-10-25 10:00:00, 2023-10-25 15:00:00}|[72.25, 71.8]        |-0.45000000000000284|
|MS  |{2023-10-25 10:00:00, 2023-10-25 15:00:00}|[23.85, 21.83]       |-2.020000000000003  |
|PGR |{2023-10-25 10:00:00, 2023-10-25 15:00:00}|[25.68, 25.14]       |-0.5399999999999991 |
|NFLX|{2023-10-25 10:00:00, 2023-10-25 15:00:00}|[28.1514, 26.0499]   |-2.101499999999998  |
|AKAM|{2023-10-25 10:00:00, 2023-10-25 15:00:00}|[38.76, 35.3099]     |-3.450099999999999  |
|DAL |{2023-10-25 10:00:00, 2023-10-25 15:00:00}|[17.04, 15.24, 15.69]|-1.3499999999999996 |
|XYL |{2023-10-25 10:00:00, 2023-10-25 15:00:00}|[28.05, 27.59]   

-------------------------------------------
Batch: 2
-------------------------------------------
+----+------------------------------------------+------------------------------+-------------------+
|name|window                                    |prices                        |price_diff         |
+----+------------------------------------------+------------------------------+-------------------+
|VMC |{2023-10-25 10:00:00, 2023-10-25 15:00:00}|[49.91, 48.24, 49.66]         |-0.25              |
|SYK |{2023-10-25 10:00:00, 2023-10-25 15:00:00}|[65.06, 64.8]                 |-0.2600000000000051|
|WDC |{2023-10-25 10:00:00, 2023-10-25 15:00:00}|[51.0, 50.13]                 |-0.8699999999999974|
|BXP |{2023-10-25 10:00:00, 2023-10-25 15:00:00}|[110.1, 109.22]               |-0.8799999999999955|
|PXD |{2023-10-25 10:00:00, 2023-10-25 15:00:00}|[124.69, 128.0, 114.075]      |-10.614999999999995|
|ISRG|{2023-10-25 10:00:00, 2023-10-25 15:00:00}|[192.6465, 160.6632, 168.1665]|-24.47999999999

-------------------------------------------
Batch: 3
-------------------------------------------
+----+------------------------------------------+-----------------------------+--------------------+
|name|window                                    |prices                       |price_diff          |
+----+------------------------------------------+-----------------------------+--------------------+
|EQIX|{2023-10-25 10:00:00, 2023-10-25 15:00:00}|[219.94, 194.854]            |-25.085999999999984 |
|CCL |{2023-10-25 10:00:00, 2023-10-25 15:00:00}|[33.95, 32.35]               |-1.6000000000000014 |
|INCY|{2023-10-25 10:00:00, 2023-10-25 15:00:00}|[23.0, 20.71]                |-2.289999999999999  |
|CAT |{2023-10-25 10:00:00, 2023-10-25 15:00:00}|[89.55, 84.77]               |-4.780000000000001  |
|ZBH |{2023-10-25 10:00:00, 2023-10-25 15:00:00}|[81.185, 78.77]              |-2.4150000000000063 |
|COP |{2023-10-25 10:00:00, 2023-10-25 15:00:00}|[61.92, 61.72]               |-0.2000000000000

...

-------------------------------------------
Batch: 34
-------------------------------------------
+-----+------------------------------------------+--------------------------------------------------------------------------------------------------+-------------------+
|name |window                                    |prices                                                                                            |price_diff         |
+-----+------------------------------------------+--------------------------------------------------------------------------------------------------+-------------------+
|OKE  |{2023-10-25 10:00:00, 2023-10-25 15:00:00}|[53.995, 61.355, 59.57, 50.66, 38.819, 39.46, 39.07, 21.0]                                        |-32.995            |
|KMX  |{2023-10-25 10:00:00, 2023-10-25 15:00:00}|[46.9, 52.73, 51.41, 47.41, 65.02, 45.94]                                                         |-0.9600000000000009|
|XLNX |{2023-10-25 10:00:00, 2023-10-25 15:00:00}|[4

-------------------------------------------
Batch: 35
-------------------------------------------
+----+------------------------------------------+-------------------------------------------------------------------------------------------------+--------------------+
|name|window                                    |prices                                                                                           |price_diff          |
+----+------------------------------------------+-------------------------------------------------------------------------------------------------+--------------------+
|HCP |{2023-10-25 10:00:00, 2023-10-25 15:00:00}|[47.08, 53.71, 46.78, 41.87, 41.89, 43.86, 38.44, 36.33, 28.55]                                  |-18.529999999999998 |
|WDC |{2023-10-25 10:00:00, 2023-10-25 15:00:00}|[51.0, 50.13, 79.8, 87.15, 86.3, 91.5, 105.68, 98.94, 44.74]                                     |-6.259999999999998  |
|CSX |{2023-10-25 10:00:00, 2023-10-25 15:00:00}|[28.13, 

-------------------------------------------
Batch: 36
-------------------------------------------
+-----+------------------------------------------+---------------------------------------------------------------------------------------+--------------------+
|name |window                                    |prices                                                                                 |price_diff          |
+-----+------------------------------------------+---------------------------------------------------------------------------------------+--------------------+
|KSU  |{2023-10-25 10:00:00, 2023-10-25 15:00:00}|[109.11, 113.24, 104.25, 126.005, 98.35, 83.97]                                        |-25.14              |
|DISCA|{2023-10-25 10:00:00, 2023-10-25 15:00:00}|[79.95, 82.82, 82.45, 76.17, 80.66, 32.68, 33.075, 25.385, 27.88]                      |-52.07000000000001  |
|EXC  |{2023-10-25 10:00:00, 2023-10-25 15:00:00}|[37.22, 30.71, 36.495, 31.525, 35.69, 34.275, 30.71,

-------------------------------------------
Batch: 37
-------------------------------------------
+-----+------------------------------------------+-------------------------------------------------------------------------------------------------------+-------------------+
|name |window                                    |prices                                                                                                 |price_diff         |
+-----+------------------------------------------+-------------------------------------------------------------------------------------------------------+-------------------+
|HCP  |{2023-10-25 10:00:00, 2023-10-25 15:00:00}|[47.08, 53.71, 46.78, 41.87, 41.89, 43.86, 38.44, 36.33, 28.55, 35.36]                                 |-11.719999999999999|
|DE   |{2023-10-25 10:00:00, 2023-10-25 15:00:00}|[89.6, 87.6, 84.98, 89.45, 89.61, 94.285, 96.89, 96.46, 76.67, 76.94]                                  |-12.659999999999997|
|WDC  |{2023-10-25 10:00:00

-------------------------------------------
Batch: 38
-------------------------------------------
+----+------------------------------------------+--------------------------------------------------------------------------------------------+-------------------+
|name|window                                    |prices                                                                                      |price_diff         |
+----+------------------------------------------+--------------------------------------------------------------------------------------------+-------------------+
|PBCT|{2023-10-25 10:00:00, 2023-10-25 15:00:00}|[16.34, 16.505, 15.28]                                                                      |-1.0600000000000005|
|ETN |{2023-10-25 10:00:00, 2023-10-25 15:00:00}|[63.29, 70.07, 68.86, 69.8, 69.54, 53.3, 55.48, 62.13]                                      |-1.1599999999999966|
|HCN |{2023-10-25 10:00:00, 2023-10-25 15:00:00}|[74.235, 63.3, 63.96, 66.76, 62.58, 62

## 3. Select the stock that gained the most (between windows)

In [2]:
# 3. M1:

In [ ]:
from pyspark.sql.functions import window, col, last, lag
from pyspark.sql.window import Window
from pyspark.sql import DataFrame

# Function to process each micro-batch
def process_batch(df: DataFrame, epoch_id):
    # Define the window specification with partitioning by name and ordering by window
    w = Window.partitionBy("name").orderBy("window")

    # Add new columns with the lagged or previous window last value
    df_with_lag = df.withColumn("prev_last_value", lag("value").over(w)) \
                    .withColumn("previous_window", lag(col("window")).over(w))

    # Calculate the difference between the current last value and the previous last value
    df_with_diff = df_with_lag.withColumn("value_diff", col("value") - col("prev_last_value"))

    # Filter for stocks that have gained value
    stocks_gained_value = df_with_diff.filter(col("value_diff") > 0)

    # Select columns in the specified order and order by the difference in descending order
    final_df = stocks_gained_value.select(
        "name",
        "window",
        "previous_window",
        "value",
        "prev_last_value",
        "value_diff"
    ).orderBy(col("value_diff").desc())

    # Optionally, limit to top N stocks that gained the most
    top_gainers = final_df.limit(10)

    # Write the full lines to the console without truncating
    top_gainers.write.format("console").option("truncate", False).save()

# Define the window duration
window_duration = "1 minute"

# Group the data by window and stock name, and get the last value in each window
windowedLast = df \
    .withWatermark("timestamp", "2 minutes") \
    .groupBy(window(col("timestamp"), window_duration), col("name")) \
    .agg(last(col("price")).alias("value"))  # Use last value instead of avg

# Use foreachBatch to apply the custom logic to each micro-batch
query = windowedLast \
    .writeStream \
    .outputMode("update") \
    .foreachBatch(process_batch) \
    .start()

query.awaitTermination()

# note that the result is based on the task in the assignment pdf, i show all the stock that gained value, and renk them, so the first is the stock that gained the most value

23/11/06 20:06:51 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-f1dd9d64-1c11-43e1-a5ad-dc8d6c589dd9. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
23/11/06 20:06:51 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


+----+------+---------------+-----+---------------+----------+
|name|window|previous_window|value|prev_last_value|value_diff|
+----+------+---------------+-----+---------------+----------+
+----+------+---------------+-----+---------------+----------+



+----+------------------------------------------+------------------------------------------+-----+---------------+------------------+
|name|window                                    |previous_window                           |value|prev_last_value|value_diff        |
+----+------------------------------------------+------------------------------------------+-----+---------------+------------------+
|MNST|{2023-10-25 10:12:00, 2023-10-25 10:13:00}|{2023-10-25 10:11:00, 2023-10-25 10:12:00}|18.98|15.8933        |3.0867000000000004|
+----+------------------------------------------+------------------------------------------+-----+---------------+------------------+



+----+------------------------------------------+------------------------------------------+--------+---------------+------------------+
|name|window                                    |previous_window                           |value   |prev_last_value|value_diff        |
+----+------------------------------------------+------------------------------------------+--------+---------------+------------------+
|WAT |{2023-10-25 10:13:00, 2023-10-25 10:14:00}|{2023-10-25 10:12:00, 2023-10-25 10:13:00}|98.99   |88.89          |10.099999999999994|
|ISRG|{2023-10-25 10:13:00, 2023-10-25 10:14:00}|{2023-10-25 10:12:00, 2023-10-25 10:13:00}|168.1665|160.6632       |7.503300000000024 |
|STI |{2023-10-25 10:13:00, 2023-10-25 10:14:00}|{2023-10-25 10:12:00, 2023-10-25 10:13:00}|31.3    |28.29          |3.0100000000000016|
|DG  |{2023-10-25 10:13:00, 2023-10-25 10:14:00}|{2023-10-25 10:12:00, 2023-10-25 10:13:00}|53.69   |52.735         |0.9549999999999983|
+----+-----------------------------------

+----+------+---------------+-----+---------------+----------+
|name|window|previous_window|value|prev_last_value|value_diff|
+----+------+---------------+-----+---------------+----------+
+----+------+---------------+-----+---------------+----------+



...

+----+------------------------------------------+------------------------------------------+-----+---------------+-------------------+
|name|window                                    |previous_window                           |value|prev_last_value|value_diff         |
+----+------------------------------------------+------------------------------------------+-----+---------------+-------------------+
|A   |{2023-10-25 10:21:00, 2023-10-25 10:22:00}|{2023-10-25 10:20:00, 2023-10-25 10:21:00}|60.38|57.88          |2.5                |
|FISV|{2023-10-25 10:21:00, 2023-10-25 10:22:00}|{2023-10-25 10:20:00, 2023-10-25 10:21:00}|56.31|55.0           |1.3100000000000023 |
|IRM |{2023-10-25 10:21:00, 2023-10-25 10:22:00}|{2023-10-25 10:20:00, 2023-10-25 10:21:00}|28.09|27.34          |0.75               |
|PM  |{2023-10-25 10:21:00, 2023-10-25 10:22:00}|{2023-10-25 10:20:00, 2023-10-25 10:21:00}|79.5 |78.95          |0.5499999999999972 |
|IR  |{2023-10-25 10:21:00, 2023-10-25 10:22:00}|{2023-

+----+------+---------------+-----+---------------+----------+
|name|window|previous_window|value|prev_last_value|value_diff|
+----+------+---------------+-----+---------------+----------+
+----+------+---------------+-----+---------------+----------+



+----+------------------------------------------+------------------------------------------+-----+---------------+-----------------+
|name|window                                    |previous_window                           |value|prev_last_value|value_diff       |
+----+------------------------------------------+------------------------------------------+-----+---------------+-----------------+
|WDC |{2023-10-25 10:22:00, 2023-10-25 10:23:00}|{2023-10-25 10:21:00, 2023-10-25 10:22:00}|91.5 |86.3           |5.200000000000003|
+----+------------------------------------------+------------------------------------------+-----+---------------+-----------------+



+----+------------------------------------------+------------------------------------------+-----+---------------+------------------+
|name|window                                    |previous_window                           |value|prev_last_value|value_diff        |
+----+------------------------------------------+------------------------------------------+-----+---------------+------------------+
|DLR |{2023-10-25 10:23:00, 2023-10-25 10:24:00}|{2023-10-25 10:22:00, 2023-10-25 10:23:00}|56.45|54.15          |2.3000000000000043|
+----+------------------------------------------+------------------------------------------+-----+---------------+------------------+



+----+------------------------------------------+------------------------------------------+------+---------------+------------------+
|name|window                                    |previous_window                           |value |prev_last_value|value_diff        |
+----+------------------------------------------+------------------------------------------+------+---------------+------------------+
|ALLE|{2023-10-25 10:24:00, 2023-10-25 10:25:00}|{2023-10-25 10:23:00, 2023-10-25 10:24:00}|56.81 |51.84          |4.969999999999999 |
|IVZ |{2023-10-25 10:24:00, 2023-10-25 10:25:00}|{2023-10-25 10:23:00, 2023-10-25 10:24:00}|37.96 |35.44          |2.520000000000003 |
|CHRW|{2023-10-25 10:24:00, 2023-10-25 10:25:00}|{2023-10-25 10:23:00, 2023-10-25 10:24:00}|61.37 |59.35          |2.019999999999996 |
|PWR |{2023-10-25 10:24:00, 2023-10-25 10:25:00}|{2023-10-25 10:23:00, 2023-10-25 10:24:00}|34.425|33.22          |1.2049999999999983|
|PVH |{2023-10-25 10:24:00, 2023-10-25 10:25:00}|{2023-

In [1]:
# 3. M2:

In [ ]:
from pyspark.sql.functions import window, col, udf
from pyspark.sql.types import DoubleType

# Define the window and watermark duration
window_duration = "5 hours"
watermark_duration = "1 hour" 

# Apply windowing and watermarking to the DataFrame
windowedStocks = df \
    .withWatermark("timestamp", watermark_duration) \
    .groupBy(
        window(col("timestamp"), window_duration),
        col("name")
    ) \
    .agg(
        {"price": "collect_list"}  # Collects all prices in the window into a list
    ) \
    .select(
        col("name"),
        col("window"),
        col("collect_list(price)").alias("prices")
    )

# Define a UDF to calculate the difference between the first and last price in the window
def price_diff(prices):
    if len(prices) > 1:
        return prices[-1] - prices[0]  # Assuming the prices are in chronological order
    else:
        return 0.0

price_diff_udf = udf(price_diff, DoubleType())

# Apply the UDF to calculate the price difference
stocksPriceDiff = windowedStocks.withColumn("price_diff", price_diff_udf(col("prices")))

# sort by the price difference in descending order
stocksGainedMost = stocksPriceDiff.orderBy(col("price_diff").desc())

# Output the result to a console sink
query = stocksGainedMost \
    .writeStream \
    .outputMode("complete") \
    .format("console") \
    .option("truncate", False) \
    .start()

query.awaitTermination()

23/11/06 20:10:00 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-0dbebbda-79b6-404f-9f10-1b433adad859. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
23/11/06 20:10:00 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


-------------------------------------------
Batch: 0
-------------------------------------------
+----+------------------------------------------+----------------+------------------+
|name|window                                    |prices          |price_diff        |
+----+------------------------------------------+----------------+------------------+
|PXD |{2023-10-25 10:00:00, 2023-10-25 15:00:00}|[124.69, 128.0] |3.3100000000000023|
|FISV|{2023-10-25 10:00:00, 2023-10-25 15:00:00}|[40.795, 42.385]|1.5899999999999963|
|AIZ |{2023-10-25 10:00:00, 2023-10-25 15:00:00}|[42.13, 42.76]  |0.6299999999999955|
|IT  |{2023-10-25 10:00:00, 2023-10-25 15:00:00}|[48.93, 49.27]  |0.3400000000000034|
|AZO |{2023-10-25 10:00:00, 2023-10-25 15:00:00}|[384.01]        |0.0               |
|HCP |{2023-10-25 10:00:00, 2023-10-25 15:00:00}|[47.08]         |0.0               |
|SYY |{2023-10-25 10:00:00, 2023-10-25 15:00:00}|[32.21]         |0.0               |
|ARE |{2023-10-25 10:00:00, 2023-10-25 15:0

-------------------------------------------
Batch: 1
-------------------------------------------
+----+------------------------------------------+----------------------+-------------------+
|name|window                                    |prices                |price_diff         |
+----+------------------------------------------+----------------------+-------------------+
|ADBE|{2023-10-25 10:00:00, 2023-10-25 15:00:00}|[39.15, 45.17]        |6.020000000000003  |
|TGT |{2023-10-25 10:00:00, 2023-10-25 15:00:00}|[63.21, 68.83]        |5.619999999999997  |
|YUM |{2023-10-25 10:00:00, 2023-10-25 15:00:00}|[64.536, 69.9]        |5.364000000000004  |
|UHS |{2023-10-25 10:00:00, 2023-10-25 15:00:00}|[57.52, 62.59]        |5.07               |
|PXD |{2023-10-25 10:00:00, 2023-10-25 15:00:00}|[124.69, 128.0]       |3.3100000000000023 |
|MNST|{2023-10-25 10:00:00, 2023-10-25 15:00:00}|[15.8933, 18.98]      |3.0867000000000004 |
|MDLZ|{2023-10-25 10:00:00, 2023-10-25 15:00:00}|[27.67, 30.58]   

-------------------------------------------
Batch: 2
-------------------------------------------
+----+------------------------------------------+----------------------+------------------+
|name|window                                    |prices                |price_diff        |
+----+------------------------------------------+----------------------+------------------+
|VRTX|{2023-10-25 10:00:00, 2023-10-25 15:00:00}|[46.46, 77.39]        |30.93             |
|GWW |{2023-10-25 10:00:00, 2023-10-25 15:00:00}|[228.67, 254.0]       |25.330000000000013|
|MCO |{2023-10-25 10:00:00, 2023-10-25 15:00:00}|[48.96, 64.36]        |15.399999999999999|
|KMB |{2023-10-25 10:00:00, 2023-10-25 15:00:00}|[90.08, 104.92]       |14.840000000000003|
|CMG |{2023-10-25 10:00:00, 2023-10-25 15:00:00}|[367.0, 378.4]        |11.399999999999977|
|FRT |{2023-10-25 10:00:00, 2023-10-25 15:00:00}|[106.58, 116.94]      |10.36             |
|WAT |{2023-10-25 10:00:00, 2023-10-25 15:00:00}|[88.89, 98.99]        |10.

-------------------------------------------
Batch: 3
-------------------------------------------
+----+------------------------------------------+----------------------------+------------------+
|name|window                                    |prices                      |price_diff        |
+----+------------------------------------------+----------------------------+------------------+
|GWW |{2023-10-25 10:00:00, 2023-10-25 15:00:00}|[228.67, 254.0, 261.34]     |32.66999999999999 |
|VRTX|{2023-10-25 10:00:00, 2023-10-25 15:00:00}|[46.46, 77.39]              |30.93             |
|GS  |{2023-10-25 10:00:00, 2023-10-25 15:00:00}|[145.83, 154.94, 168.2]     |22.369999999999976|
|KMB |{2023-10-25 10:00:00, 2023-10-25 15:00:00}|[90.08, 104.92]             |14.840000000000003|
|MCO |{2023-10-25 10:00:00, 2023-10-25 15:00:00}|[48.96, 64.36, 62.93]       |13.969999999999999|
|CMG |{2023-10-25 10:00:00, 2023-10-25 15:00:00}|[367.0, 378.4]              |11.399999999999977|
|FRT |{2023-10-25 10:

-------------------------------------------
Batch: 4
-------------------------------------------
+----+------------------------------------------+-----------------------------------+------------------+
|name|window                                    |prices                             |price_diff        |
+----+------------------------------------------+-----------------------------------+------------------+
|GWW |{2023-10-25 10:00:00, 2023-10-25 15:00:00}|[228.67, 254.0, 261.34]            |32.66999999999999 |
|VRTX|{2023-10-25 10:00:00, 2023-10-25 15:00:00}|[46.46, 77.39]                     |30.93             |
|AGN |{2023-10-25 10:00:00, 2023-10-25 15:00:00}|[93.13, 122.38]                    |29.25             |
|KMB |{2023-10-25 10:00:00, 2023-10-25 15:00:00}|[90.08, 104.92]                    |14.840000000000003|
|MCO |{2023-10-25 10:00:00, 2023-10-25 15:00:00}|[48.96, 64.36, 62.93]              |13.969999999999999|
|EQT |{2023-10-25 10:00:00, 2023-10-25 15:00:00}|[67.3, 81.26] 

-------------------------------------------
Batch: 5
-------------------------------------------
+-----+------------------------------------------+------------------------------+------------------+
|name |window                                    |prices                        |price_diff        |
+-----+------------------------------------------+------------------------------+------------------+
|VRTX |{2023-10-25 10:00:00, 2023-10-25 15:00:00}|[46.46, 77.39, 81.34]         |34.88             |
|GWW  |{2023-10-25 10:00:00, 2023-10-25 15:00:00}|[228.67, 254.0, 261.34]       |32.66999999999999 |
|AGN  |{2023-10-25 10:00:00, 2023-10-25 15:00:00}|[93.13, 122.38]               |29.25             |
|GOOGL|{2023-10-25 10:00:00, 2023-10-25 15:00:00}|[414.2338, 453.9535, 432.8925]|18.65870000000001 |
|MCO  |{2023-10-25 10:00:00, 2023-10-25 15:00:00}|[48.96, 64.36, 62.93]         |13.969999999999999|
|EQT  |{2023-10-25 10:00:00, 2023-10-25 15:00:00}|[67.3, 81.26]                 |13.96000000000

-------------------------------------------
Batch: 6
-------------------------------------------
+-----+------------------------------------------+---------------------------------------+------------------+
|name |window                                    |prices                                 |price_diff        |
+-----+------------------------------------------+---------------------------------------+------------------+
|PCLN |{2023-10-25 10:00:00, 2023-10-25 15:00:00}|[819.98, 982.1298]                     |162.14980000000003|
|VRTX |{2023-10-25 10:00:00, 2023-10-25 15:00:00}|[46.46, 77.39, 81.34]                  |34.88             |
|GWW  |{2023-10-25 10:00:00, 2023-10-25 15:00:00}|[228.67, 254.0, 261.34]                |32.66999999999999 |
|AGN  |{2023-10-25 10:00:00, 2023-10-25 15:00:00}|[93.13, 122.38]                        |29.25             |
|GOOGL|{2023-10-25 10:00:00, 2023-10-25 15:00:00}|[414.2338, 453.9535, 432.8925, 436.436]|22.202200000000005|
|TRIP |{2023-10-25 10:0

## 4. Implement a control that checks if a stock does not lose too much value in a period of time

In [ ]:
from pyspark.sql.functions import window, col, udf, min, max, first, last
from pyspark.sql.types import DoubleType

# Define the window and watermark duration
window_duration = "5 hours"
watermark_duration = "1 hour" 

# Apply windowing and watermarking to the DataFrame
windowedStocks = df \
    .withWatermark("timestamp", watermark_duration) \
    .groupBy(
        window(col("timestamp"), window_duration),
        col("name")
    ) \
    .agg(
        first("price").alias("first_price"),
        last("price").alias("last_price")
    )

# Define a UDF to calculate the percentage change
def price_drop_percentage(first_price, last_price):
    if first_price is not None and last_price is not None and first_price != 0:
        return ((last_price - first_price) / first_price) * 100
    else:
        return None

price_drop_percentage_udf = udf(price_drop_percentage, DoubleType())

# Apply the UDF to calculate the percentage change
stocksPriceChange = windowedStocks.withColumn("price_drop_perc", price_drop_percentage_udf(col("first_price"), col("last_price")))

# Filter the stocks that lost too much value
stocksLostTooMuchValue = stocksPriceChange.filter(col("price_drop_perc") <= -10) # 10% of the value is a critical value

# Output the result to a console sink
query = stocksLostTooMuchValue \
    .writeStream \
    .outputMode("update") \
    .format("console") \
    .option("truncate", False) \
    .start()

query.awaitTermination()

# price_drop_perc is the percentage of value that the stock lost

23/11/06 20:00:42 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-822d5286-1276-4898-ad9f-a0558bf42e28. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
23/11/06 20:00:42 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


-------------------------------------------
Batch: 0
-------------------------------------------
+------+----+-----------+----------+---------------+
|window|name|first_price|last_price|price_drop_perc|
+------+----+-----------+----------+---------------+
+------+----+-----------+----------+---------------+



-------------------------------------------
Batch: 1
-------------------------------------------
+------+----+-----------+----------+---------------+
|window|name|first_price|last_price|price_drop_perc|
+------+----+-----------+----------+---------------+
+------+----+-----------+----------+---------------+



-------------------------------------------
Batch: 2
-------------------------------------------
+------------------------------------------+----+-----------+----------+-------------------+
|window                                    |name|first_price|last_price|price_drop_perc    |
+------------------------------------------+----+-----------+----------+-------------------+
|{2023-10-25 10:00:00, 2023-10-25 15:00:00}|ISRG|192.6465   |168.1665  |-12.707212433135295|
+------------------------------------------+----+-----------+----------+-------------------+



-------------------------------------------
Batch: 3
-------------------------------------------
+------------------------------------------+----+-----------+----------+-------------------+
|window                                    |name|first_price|last_price|price_drop_perc    |
+------------------------------------------+----+-----------+----------+-------------------+
|{2023-10-25 10:00:00, 2023-10-25 15:00:00}|EQIX|219.94     |194.854   |-11.405837955806122|
|{2023-10-25 10:00:00, 2023-10-25 15:00:00}|EXC |37.22      |30.71     |-17.490596453519608|
+------------------------------------------+----+-----------+----------+-------------------+



...

-------------------------------------------
Batch: 5
-------------------------------------------
+------------------------------------------+----+-----------+----------+-------------------+
|window                                    |name|first_price|last_price|price_drop_perc    |
+------------------------------------------+----+-----------+----------+-------------------+
|{2023-10-25 10:00:00, 2023-10-25 15:00:00}|DHI |24.5       |18.91     |-22.816326530612244|
|{2023-10-25 10:00:00, 2023-10-25 15:00:00}|MPC |42.505     |35.945    |-15.433478414304206|
|{2023-10-25 10:00:00, 2023-10-25 15:00:00}|PHM |19.69      |16.11     |-18.18181818181819 |
+------------------------------------------+----+-----------+----------+-------------------+



-------------------------------------------
Batch: 6
-------------------------------------------
+------------------------------------------+----+-----------+----------+-------------------+
|window                                    |name|first_price|last_price|price_drop_perc    |
+------------------------------------------+----+-----------+----------+-------------------+
|{2023-10-25 10:00:00, 2023-10-25 15:00:00}|HCP |47.08      |41.87     |-11.066270178419714|
|{2023-10-25 10:00:00, 2023-10-25 15:00:00}|KO  |43.05      |38.61     |-10.313588850174211|
|{2023-10-25 10:00:00, 2023-10-25 15:00:00}|LEN |36.2157    |31.5784   |-12.804667588918617|
+------------------------------------------+----+-----------+----------+-------------------+



-------------------------------------------
Batch: 7
-------------------------------------------
+------------------------------------------+----+-----------+----------+-------------------+
|window                                    |name|first_price|last_price|price_drop_perc    |
+------------------------------------------+----+-----------+----------+-------------------+
|{2023-10-25 10:00:00, 2023-10-25 15:00:00}|HCN |74.235     |63.96     |-13.841180036370982|
|{2023-10-25 10:00:00, 2023-10-25 15:00:00}|T   |37.69      |33.65     |-10.719023613690633|
|{2023-10-25 10:00:00, 2023-10-25 15:00:00}|SPG |175.05     |151.49    |-13.45901171093973 |
|{2023-10-25 10:00:00, 2023-10-25 15:00:00}|CAG |35.64      |31.02     |-12.962962962962965|
|{2023-10-25 10:00:00, 2023-10-25 15:00:00}|SCG |53.2375    |47.5      |-10.77717774125381 |
|{2023-10-25 10:00:00, 2023-10-25 15:00:00}|MAA |74.94      |63.25     |-15.599145983453427|
+------------------------------------------+----+-----------+-----

-------------------------------------------
Batch: 8
-------------------------------------------
+------------------------------------------+----+-----------+----------+-------------------+
|window                                    |name|first_price|last_price|price_drop_perc    |
+------------------------------------------+----+-----------+----------+-------------------+
|{2023-10-25 10:00:00, 2023-10-25 15:00:00}|HCN |74.235     |66.76     |-10.069374284367205|
+------------------------------------------+----+-----------+----------+-------------------+



-------------------------------------------
Batch: 9
-------------------------------------------
+------------------------------------------+----+-----------+----------+-------------------+
|window                                    |name|first_price|last_price|price_drop_perc    |
+------------------------------------------+----+-----------+----------+-------------------+
|{2023-10-25 10:00:00, 2023-10-25 15:00:00}|NEM |39.43      |26.24     |-33.451686533096634|
|{2023-10-25 10:00:00, 2023-10-25 15:00:00}|DLR |64.3965    |46.315    |-28.07838935345866 |
|{2023-10-25 10:00:00, 2023-10-25 15:00:00}|VTR |72.1       |59.44     |-17.55894590846047 |
|{2023-10-25 10:00:00, 2023-10-25 15:00:00}|IBM |211.98     |186.2     |-12.161524672138881|
+------------------------------------------+----+-----------+----------+-------------------+



## 5. Asset changes with the fluctuation of the market

In [ ]:
from pyspark.sql import functions as F

# Define the stocks you own
owned_stocks = [
    ('CSCO', 100),  # 100 shares of CSCO
    ('ALGN', 50),   # 50 shares of ALGN
    ('NKE', 150),   # 150 shares of NKE
    ('ESS', 70)     # 70 shares of ESS
]

# Create a DataFrame for the owned stocks
owned_stocks_df = spark.createDataFrame(owned_stocks, ['name', 'amount'])

# Show the DataFrame
owned_stocks_df.show()

# Join the streaming DataFrame with the owned stocks DataFrame
asset_value = df.join(owned_stocks_df, "name", "inner")

# Calculate the value of each owned stock
asset_value = asset_value.withColumn("asset_value", F.col("price") * F.col("amount"))

# Group by the stock name and apply a window function to calculate the fluctuation over a 5-hour window
windowed_asset_value = asset_value \
    .groupBy(F.window(F.col("timestamp"), "5 hours"), F.col("name")) \
    .agg(F.sum("asset_value").alias("total_value"))

# Start the streaming query to calculate the asset value over time
query = windowed_asset_value \
    .writeStream \
    .outputMode("complete") \
    .format("console") \
    .start()

query.awaitTermination()

+----+------+
|name|amount|
+----+------+
|CSCO|   100|
|ALGN|    50|
| NKE|   150|
| ESS|    70|
+----+------+



23/11/06 22:30:50 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-2bd5edfd-e2b2-4c7f-be1e-860b22e03570. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
23/11/06 22:30:50 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


-------------------------------------------
Batch: 0
-------------------------------------------
+------+----+-----------+
|window|name|total_value|
+------+----+-----------+
+------+----+-----------+



-------------------------------------------
Batch: 1
-------------------------------------------
+------+----+-----------+
|window|name|total_value|
+------+----+-----------+
+------+----+-----------+



-------------------------------------------
Batch: 2
-------------------------------------------
+--------------------+----+------------------+
|              window|name|       total_value|
+--------------------+----+------------------+
|{2023-10-25 10:00...|ALGN|1852.4999999999998|
|{2023-10-25 10:00...| ESS|10887.449999999999|
|{2023-10-25 10:00...| NKE|            4948.5|
+--------------------+----+------------------+



...

-------------------------------------------
Batch: 8
-------------------------------------------
+--------------------+----+-----------+
|              window|name|total_value|
+--------------------+----+-----------+
|{2023-10-25 10:00...|ALGN|     3789.5|
|{2023-10-25 10:00...| ESS|    54747.0|
|{2023-10-25 10:00...| NKE|   10235.25|
|{2023-10-25 10:00...|CSCO|     2469.0|
+--------------------+----+-----------+



-------------------------------------------
Batch: 9
-------------------------------------------
+--------------------+----+-----------+
|              window|name|total_value|
+--------------------+----+-----------+
|{2023-10-25 10:00...|ALGN|     3789.5|
|{2023-10-25 10:00...| ESS|    54747.0|
|{2023-10-25 10:00...| NKE|   10235.25|
|{2023-10-25 10:00...|CSCO|     2469.0|
+--------------------+----+-----------+



-------------------------------------------
Batch: 10
-------------------------------------------
+--------------------+----+-----------+
|              window|name|total_value|
+--------------------+----+-----------+
|{2023-10-25 10:00...|ALGN|     3789.5|
|{2023-10-25 10:00...| ESS|    54747.0|
|{2023-10-25 10:00...| NKE|   15830.25|
|{2023-10-25 10:00...|CSCO|     2469.0|
+--------------------+----+-----------+



-------------------------------------------
Batch: 11
-------------------------------------------
+--------------------+----+-----------+
|              window|name|total_value|
+--------------------+----+-----------+
|{2023-10-25 10:00...|ALGN|     3789.5|
|{2023-10-25 10:00...| ESS|    54747.0|
|{2023-10-25 10:00...| NKE|   15830.25|
|{2023-10-25 10:00...|CSCO|     2469.0|
+--------------------+----+-----------+



-------------------------------------------
Batch: 12
-------------------------------------------
+--------------------+----+-----------+
|              window|name|total_value|
+--------------------+----+-----------+
|{2023-10-25 10:00...|ALGN|     6526.5|
|{2023-10-25 10:00...| ESS|    54747.0|
|{2023-10-25 10:00...| NKE|   15830.25|
|{2023-10-25 10:00...|CSCO|     2469.0|
+--------------------+----+-----------+

